# [TEST] Cleaning CSV Master

# 0. Libraries and Packages

In [1]:
import numpy as np
import pandas as pd

In [10]:
file_path = '/Users/salmadanu/Desktop/Skripsi/skripsi-env/narasipal/master_csv/kompas_master.csv'
df = pd.read_csv(file_path)
df.head()

,content_PublishedDate,content_editor,content_site,content_tags,content_title,content_total_words,content_type,url
0,"23 November 2023, 18:17 WIB",Adinda Septia Berliana,Kompas.com,unknown,Israel Tangkap Direktur RS Al Shifa di Gaza,unknown,Video,https://video.kompas.com/watch/1049303/israel-...
1,"23 November 2023, 17:17 WIB",unknown,Kompas.com,unknown,Tentara Israel Masuki Terowongan Hamas di RS A...,unknown,Video,https://video.kompas.com/watch/1049125/tentara...
2,"23 November 2023, 16:21 WIB",unknown,Kompas.com,unknown,Rencana Erdogan Usai Hamas dan Israel Sepakat ...,unknown,Video,https://video.kompas.com/watch/1048935/erdogan...
3,"23 November 2023, 16:18 WIB",Vina Muthi Ambarwati,Kompas.com,unknown,"Israel-Hamas Gencatan Senjata, Hizbullah Tetap...",unknown,Video,https://video.kompas.com/watch/1048929/israel-...
4,"23 November 2023, 16:17 WIB",Bernadetha Nadia Deni Ananda,Kompas.com,unknown,"Menlu Iran Bertemu Pemimpin Hamas, Bahas Gaza",unknown,Video,https://video.kompas.com/watch/1048928/menlu-i...


# 1. Cleaning Data
### Format:
- `id` = `int`
- `content_PublishedDate` = `datetime` YYYY-MM-DD HH:MM:SS
- `content_editor` = `object`
- `content_site` = `object `Kompas.com
- `content_tags` = `str`
- `content_title` = `str`
- `content_total_words` = `int`
- `content_type` = `object`
- `url` = `int`

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1258 entries, 0 to 1257
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   content_PublishedDate  983 non-null    object
 1   content_editor         983 non-null    object
 2   content_site           983 non-null    object
 3   content_tags           983 non-null    object
 4   content_title          983 non-null    object
 5   content_total_words    983 non-null    object
 6   content_type           983 non-null    object
 7   url                    1258 non-null   object
dtypes: object(8)
memory usage: 78.8+ KB


## Checking for duplicate rows

In [12]:
duplicate = df[df.duplicated]
duplicate.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21 entries, 549 to 957
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   content_PublishedDate  21 non-null     object
 1   content_editor         21 non-null     object
 2   content_site           21 non-null     object
 3   content_tags           21 non-null     object
 4   content_title          21 non-null     object
 5   content_total_words    21 non-null     object
 6   content_type           21 non-null     object
 7   url                    21 non-null     object
dtypes: object(8)
memory usage: 1.5+ KB


In [13]:
df_clean = df.drop_duplicates()

In [14]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1237 entries, 0 to 1257
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   content_PublishedDate  962 non-null    object
 1   content_editor         962 non-null    object
 2   content_site           962 non-null    object
 3   content_tags           962 non-null    object
 4   content_title          962 non-null    object
 5   content_total_words    962 non-null    object
 6   content_type           962 non-null    object
 7   url                    1237 non-null   object
dtypes: object(8)
memory usage: 87.0+ KB


## Standardizing `content_PublishedDate` and extracting `content_title` from `url`
YYYY-MM-DD HH:MM:SS

Logic (use regex)
for all rows in df
if `content_PublishedDate` has value:
    if format is not YYYY-MM-DD HH:MM:SS
        change to YYYY-MM-DD HH:MM:SS based on:
            4 number sequence = YYYY
            MM ikutin mapping:
                January = 01
                February = 02
            add space
            2 number sequence = DD
            erase ,
            erase WIB
            HH:MM to HH:SS:00
    else:
        keep format
else:
    null


if `content_title` is NULL:
    df[content_title] = last part of url (all text from the end before the last '/')
    remove -
    make title case

In [15]:
import pandas as pd
import re
from urllib.parse import urlparse

In [16]:
month_map = {
    "January": "01", 
    "February": "02", 
    "March": "03", 
    "April": "04",
    "May": "05", 
    "June": "06", 
    "July": "07", 
    "August": "08",
    "September": "09", 
    "October": "10", 
    "November": "11", 
    "December": "12"
}

In [ ]:
df_clean = df.copy()

# Funtion for standardizing datetime
def standardize_date(date_str):
    if pd.isna(date_str):
        return None
    match = re.search(r"(\d{1,2}) (\w+) (\d{4}), (\d{2}):(\d{2}) WIB", date_str)
    if match:
        day, month, year, hour, minute = match.groups()
        return f"{year}-{month_map.get(month, 'XX')}-{int(day):02d} {hour}:{minute}:00"
    return date_str 

# Funtion for extracting judul berita from url
def extract_title_from_url(url):
    if pd.isna(url):
        return None
    path = urlparse(url).path
    last_part = path.rstrip("/").split("/")[-1]
    return last_part.replace("-", " ").title()

# Apply transformations
df_clean.loc[:, "content_PublishedDate"] = df_clean["content_PublishedDate"].apply(standardize_date)
df_clean.loc[:, "content_title"] = df_clean.apply(
    lambda row: extract_title_from_url(row["url"]) if pd.isna(row["content_title"]) else row["content_title"], axis=1
)

df_clean


,content_PublishedDate,content_editor,content_site,content_tags,content_title,content_total_words,content_type,url
0,2023-11-23 18:17:00,Adinda Septia Berliana,Kompas.com,unknown,Israel Tangkap Direktur RS Al Shifa di Gaza,unknown,Video,https://video.kompas.com/watch/1049303/israel-...
1,2023-11-23 17:17:00,unknown,Kompas.com,unknown,Tentara Israel Masuki Terowongan Hamas di RS A...,unknown,Video,https://video.kompas.com/watch/1049125/tentara...
2,2023-11-23 16:21:00,unknown,Kompas.com,unknown,Rencana Erdogan Usai Hamas dan Israel Sepakat ...,unknown,Video,https://video.kompas.com/watch/1048935/erdogan...
3,2023-11-23 16:18:00,Vina Muthi Ambarwati,Kompas.com,unknown,"Israel-Hamas Gencatan Senjata, Hizbullah Tetap...",unknown,Video,https://video.kompas.com/watch/1048929/israel-...
4,2023-11-23 16:17:00,Bernadetha Nadia Deni Ananda,Kompas.com,unknown,"Menlu Iran Bertemu Pemimpin Hamas, Bahas Gaza",unknown,Video,https://video.kompas.com/watch/1048928/menlu-i...
...,...,...,...,...,...,...,...,...
1253,2023-11-29 17:15:00,unknown,Kompas.com,unknown,Momen Sandera Hamas Lambaikan Tangannya Saat D...,unknown,Video,https://video.kompas.com/watch/1062854/momen-s...
1254,2023-11-29 17:14:00,Aditya Jaya Iswara,Kompas.com,"sejarah Tepi Barat, sejarah wilayah Tepi Barat...",Sejarah Wilayah Tepi Barat dalam Konflik Israe...,2158,Standard,http://internasional.kompas.com/read/2023/11/2...
1255,2023-11-29 17:13:00,unknown,Kompas.com,unknown,"Dari Podium Sidang PBB, Menlu Retno Suarakan K...",unknown,Video,https://video.kompas.com/watch/1062853/dari-po...
1256,2023-11-29 17:09:49,Dani Prabowo,Kompas.com,"Israel, Palestina, pemilu, Bentrok di Bitung","Pasca Bentrokan di Bitung, Kapolri Minta Masya...",261,Standard,http://nasional.kompas.com/read/2023/11/29/170...


## New dataframe `df_title` with just `content_title`

In [22]:
df_title = df_clean[['content_title']]
df_title.head()

,content_title
0,Israel Tangkap Direktur RS Al Shifa di Gaza
1,Tentara Israel Masuki Terowongan Hamas di RS A...
2,Rencana Erdogan Usai Hamas dan Israel Sepakat ...
3,"Israel-Hamas Gencatan Senjata, Hizbullah Tetap..."
4,"Menlu Iran Bertemu Pemimpin Hamas, Bahas Gaza"


## `df_title` to CSV

In [24]:
df_title.to_csv('/Users/salmadanu/Desktop/Skripsi/skripsi-env/narasipal/cleaning/title_csv/kompas_titles.csv', index=True)